In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!git clone https://github.com/samar-khanna/DiffusionSat/

Cloning into 'DiffusionSat'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 584 (delta 1), reused 0 (delta 0), pack-reused 582 (from 1)
Receiving objects: 100% (584/584), 56.78 MiB | 13.75 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
%%bash
cd DiffusionSat

# if you want cuda 11.8, replace the index url with https://download.pytorch.org/whl/cu118
pip install torch==2.2.2 torchvision==0.17.2 --index-url https://download.pytorch.org/whl/cu121
pip install -e ".[torch]"  # install editable diffusers
pip install -r requirements_remaining.txt

Looking in indexes: https://download.pytorch.org/whl/cu121
Obtaining file:///content/DiffusionSat
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for diffusers (pyproject.toml): started
  Building editable for diffusers (pyproject.toml): finished with status 'done'
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-0.editable-py3-none-any.whl size=10017 sha256=f662d2f0b906c4764bc6b071842ceff0300e69e47c9db92b3873a4cd9f69d061
  Stored in directory: /tmp/pip-ephem-wheel-cache-t_0_1d4s/wheels/1f/30/f6/5128acdd8e757c3a37

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


CalledProcessError: Command 'b'cd DiffusionSat\n\n# if you want cuda 11.8, replace the index url with https://download.pytorch.org/whl/cu118\npip install torch==2.2.2 torchvision==0.17.2 --index-url https://download.pytorch.org/whl/cu121\npip install -e ".[torch]"  # install editable diffusers\npip install -r requirements_remaining.txt\n'' returned non-zero exit status 1.

In [ ]:

import sys
sys.path.append('..')

import argparse
import logging
import math
import os
import random
import webdataset as wds
from pathlib import Path
from copy import deepcopy
import pandas as pd
import numpy as np
from PIL import Image

from pyproj import Geod
from shapely.geometry import shape as shapey
from shapely.wkt import loads as shape_loads

import torch
from torchvision import transforms

from diffusionsat import (
    SatUNet, DiffusionSatPipeline,
    SampleEqually,
    fmow_tokenize_caption, fmow_numerical_metadata,
    spacenet_tokenize_caption, spacenet_numerical_metadata,
    satlas_tokenize_caption, satlas_numerical_metadata,
    combine_text_and_metadata, metadata_normalize,
)

In [ ]:

# cache location
os.environ["HF_HOME"] = "path/to/.cache/"

Caption/metadata

In [ ]:
caption = "a nadir UAV-style aerial photograph of an urban area, realistic colours, natural lighting, fine details, realistic textures"
metadata = metadata_normalize([76.5712666476, 28.6965307997, 0.929417550564, 0.0765712666476, 2015, 2, 27]).tolist()

Pipe with No Finetuning

In [ ]:

unet1 = SatUNet.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="unet", use_metadata=False, torch_dtype=torch.float16)
pipe1 = DiffusionSatPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", unet=unet1, torch_dtype=torch.float16)
pipe1 = pipe1.to("cuda")

In [ ]:

image = pipe1(caption, num_inference_steps=50, guidance_scale=7.5, height=512, width=512).images[0]
display(image)